# Config Tuning: Bolinas Surf Forecast

Load CDIP MOP MA147 hindcast data and compare how config parameter changes
affect swell scores and surf height predictions across a range of real conditions.

**Workflow:**
1. Load hindcast and baseline config
2. Auto-select representative days (big/small swell, clean/messy, long/short period)
3. Compute baseline scores
4. Modify config values in the Experiment cell
5. Compare via table and plots

In [1]:
import sys
from pathlib import Path
import copy
import yaml
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

ROOT = Path.cwd().parent if Path.cwd().name == "simulations" else Path.cwd()
sys.path.insert(0, str(ROOT / "src"))

from process_data import compute_swell_score, predict_bolinas_surf_height

## 1. Load Hindcast Data

Pulls MA147 hindcast from CDIP THREDDS, filtered to the swell frequency band.

In [2]:
HINDCAST_URL = (
    "https://thredds.cdip.ucsd.edu/thredds/dodsC/"
    "cdip/model/MOP_alongshore/MA147_hindcast.nc"
)

ds = xr.open_dataset(HINDCAST_URL, engine="netcdf4")
ds_swell = ds.sel(waveFrequency=ds.waveFrequency[ds.waveFrequency <= 0.12]).load()

print(f"Loaded {len(ds_swell.waveTime)} timesteps")
print(f"Time range: {pd.Timestamp(ds_swell.waveTime.values[0])} to {pd.Timestamp(ds_swell.waveTime.values[-1])}")

Loaded 221328 timesteps
Time range: 2000-01-01 00:00:00 to 2025-03-31 23:00:00


## 2. Load Baseline Config

In [3]:
with open(ROOT / "config" / "surf_config.yaml", "r") as f:
    baseline_config = yaml.safe_load(f)

baseline_surf = baseline_config["surf_model"]
print("Spectral scoring:", baseline_surf["spectral_scoring"])
print("Nearshore:", baseline_surf["nearshore"])

Spectral scoring: {'hs_min_m': 0.3, 'hs_full_credit_m': 1.5, 'tp_min_s': 10.0, 'tp_full_credit_s': 16.0, 'spread_min_deg': 5.0, 'spread_max_deg': 20.0, 'w_hs': 0.4, 'w_tp': 0.4, 'w_sp': 0.2}
Nearshore: {'range_factor': 0.15, 'range_period_min': 12, 'range_step': 0.01}


## 3. Compute Baseline Scores

In [4]:
def score_with_config(ds, surf_cfg):
    """Compute swell scores and surf heights for a given config."""
    scores = compute_swell_score(ds, surf_cfg)
    heights = scores.apply(
        lambda row: predict_bolinas_surf_height(
            row["hs_swell"] * 3.28084, row["tp"], surf_cfg["nearshore"]
        ),
        axis=1,
    )
    return pd.concat([scores, pd.DataFrame(heights.tolist(), index=scores.index)], axis=1)

baseline_scores = score_with_config(ds_swell, baseline_surf)
baseline_scores[["swell_score", "hs_swell", "tp", "spread", "bolinas_surf_min_ft", "bolinas_surf_max_ft"]].head(10)

,swell_score,hs_swell,tp,spread,bolinas_surf_min_ft,bolinas_surf_max_ft
time,,,,,,
2000-01-01 00:00:00,2.212003,0.375544,10.000000,17.427141,1.0,1.5
2000-01-01 01:00:00,2.174660,0.369147,10.000000,17.412550,1.0,1.5
2000-01-01 02:00:00,2.087209,0.361445,10.000000,17.728409,1.0,1.5
2000-01-01 03:00:00,1.990573,0.355029,10.000000,18.169527,1.0,1.5
2000-01-01 04:00:00,1.984012,0.353304,9.090909,18.122740,1.0,1.5
2000-01-01 05:00:00,2.250864,0.364203,9.090909,16.515305,1.0,1.5
2000-01-01 06:00:00,2.672481,0.399935,9.090909,14.720095,1.0,1.5
2000-01-01 07:00:00,2.887725,0.421252,9.090909,13.805140,1.0,1.5
2000-01-01 08:00:00,2.949680,0.432497,9.090909,13.721259,1.0,1.5


## 4. Select Representative Days

Auto-selects days from the hindcast that cover a range of conditions:
- Biggest swell day
- Smallest swell day (still active)
- Longest period day
- Shortest period day
- Cleanest swell (narrowest spread)
- Messiest swell (widest spread)
- Highest scoring day
- Lowest scoring day

In [5]:
# Compute daily summaries from baseline
daily = baseline_scores.copy()
daily["date"] = daily.index.date

daily_agg = daily.groupby("date").agg(
    avg_score=("swell_score", "mean"),
    max_hs=("hs_swell", "max"),
    avg_tp=("tp", "mean"),
    avg_spread=("spread", "mean"),
    avg_surf_min=("bolinas_surf_min_ft", "mean"),
    avg_surf_max=("bolinas_surf_max_ft", "mean"),
)

# Filter out flat days (no swell at all)
active = daily_agg[daily_agg["max_hs"] > 0.1]

# Pick representative days across the full range of conditions
picks = {}

# Extremes
picks["Biggest swell"] = active["max_hs"].idxmax()
picks["Smallest swell"] = active["max_hs"].idxmin()
picks["Longest period"] = active["avg_tp"].idxmax()
picks["Shortest period"] = active["avg_tp"].idxmin()
picks["Cleanest (narrow spread)"] = active["avg_spread"].idxmin()
picks["Messiest (wide spread)"] = active["avg_spread"].idxmax()
picks["Highest score"] = active["avg_score"].idxmax()
picks["Lowest score"] = active["avg_score"].idxmin()

# Average / typical days (closest to median for each metric)
def nearest_to_median(series):
    med = series.median()
    return (series - med).abs().idxmin()

picks["Avg swell size"] = nearest_to_median(active["max_hs"])
picks["Avg period"] = nearest_to_median(active["avg_tp"])
picks["Avg score"] = nearest_to_median(active["avg_score"])
picks["Avg spread"] = nearest_to_median(active["avg_spread"])

# Deduplicate (same day might win multiple categories)
seen = set()
unique_picks = {}
for label, date in picks.items():
    if date not in seen:
        unique_picks[label] = date
        seen.add(date)

rep_days = daily_agg.loc[list(unique_picks.values())].copy()
rep_days.insert(0, "condition", list(unique_picks.keys()))
rep_days = rep_days.sort_values("avg_score", ascending=False)

print(f"Selected {len(rep_days)} representative days:\n")
rep_days


Selected 11 representative days:



,condition,avg_score,max_hs,avg_tp,avg_spread,avg_surf_min,avg_surf_max
date,,,,,,,
2003-01-07,Cleanest (narrow spread),9.850831,3.574147,19.545454,6.243074,7.229167,11.437500
2020-12-03,Longest period,9.165693,1.778847,21.643518,8.013818,3.187500,5.208333
2023-01-05,Biggest swell,8.394661,6.418896,14.840068,8.415290,10.729167,16.020833
2015-09-08,Avg swell size,7.340171,0.788782,16.268454,13.796763,1.895833,2.708333
2017-04-01,Avg period,7.052864,1.073098,13.313492,11.613475,2.333333,3.229167
2009-09-15,Avg spread,6.377312,0.999217,13.010323,12.441028,2.187500,3.062500
2005-12-01,Avg score,6.240456,1.264254,11.492896,10.682633,2.500000,3.416667
2008-01-28,Messiest (wide spread),4.062851,1.277053,9.924243,20.322065,3.104167,4.208333
2000-01-21,Shortest period,2.977244,0.608869,9.090909,14.023133,1.229167,1.750000


## 5. Experiment: Modify Config

Edit the values below, then re-run this cell and the cells that follow.

Common things to try:
- `hs_min_m`: raise to penalize small swells more (default 0.3)
- `hs_full_credit_m`: lower to give full marks to smaller waves (default 1.5)
- `tp_min_s` / `tp_full_credit_s`: shift the period scoring curve
- `spread_max_deg`: raise to be more tolerant of messy swells (default 20)
- `w_hs` / `w_tp` / `w_sp`: rebalance the scoring weights (must sum to 1.0)
- `range_factor`: widen/narrow the surf height min-max spread (default 0.15)

In [12]:
experiment_config = copy.deepcopy(baseline_surf)

# --- EDIT BELOW ---
experiment_config["spectral_scoring"]["hs_min_m"] = 0.3
experiment_config["spectral_scoring"]["hs_full_credit_m"] = 1.5
experiment_config["spectral_scoring"]["tp_min_s"] = 10.0
experiment_config["spectral_scoring"]["tp_full_credit_s"] = 16.0
experiment_config["spectral_scoring"]["spread_min_deg"] = 5.0
experiment_config["spectral_scoring"]["spread_max_deg"] = 20.0
experiment_config["spectral_scoring"]["w_hs"] = 0.35
experiment_config["spectral_scoring"]["w_tp"] = 0.45
experiment_config["spectral_scoring"]["w_sp"] = 0.20
experiment_config["nearshore"]["range_factor"] = 0.15
# --- EDIT ABOVE ---

experiment_scores = score_with_config(ds_swell, experiment_config)

print(f"Baseline avg swell score:    {baseline_scores['swell_score'].mean():.2f}")
print(f"Experiment avg swell score:  {experiment_scores['swell_score'].mean():.2f}")
print(f"Delta:                       {experiment_scores['swell_score'].mean() - baseline_scores['swell_score'].mean():+.2f}")

Baseline avg swell score:    6.11
Experiment avg swell score:  6.14
Delta:                       +0.03


## 6. Day-by-Day Comparison Table

Shows how each representative day changes between baseline and experiment config.

In [13]:
# Aggregate experiment scores by day
exp_daily = experiment_scores.copy()
exp_daily["date"] = exp_daily.index.date
exp_daily_agg = exp_daily.groupby("date").agg(
    avg_score=("swell_score", "mean"),
    max_hs=("hs_swell", "max"),
    avg_tp=("tp", "mean"),
    avg_spread=("spread", "mean"),
    avg_surf_min=("bolinas_surf_min_ft", "mean"),
    avg_surf_max=("bolinas_surf_max_ft", "mean"),
)

# Build comparison table for representative days
rep_dates = rep_days.index.tolist()
base_rep = daily_agg.loc[rep_dates].copy()
exp_rep = exp_daily_agg.loc[rep_dates].copy()

comparison = pd.DataFrame({
    "Condition": rep_days["condition"].values,
    "Date": rep_dates,
    "Hs (m)": base_rep["max_hs"].round(2).values,
    "Tp (s)": base_rep["avg_tp"].round(1).values,
    "Spread": base_rep["avg_spread"].round(1).values,
    "Baseline Score": base_rep["avg_score"].round(1).values,
    "Experiment Score": exp_rep["avg_score"].round(1).values,
    "Delta": (exp_rep["avg_score"].values - base_rep["avg_score"].values).round(1),
    "Baseline Surf (ft)": (
        base_rep["avg_surf_min"].round(1).astype(str).values
        + "-"
        + base_rep["avg_surf_max"].round(1).astype(str).values
    ),
    "Experiment Surf (ft)": (
        exp_rep["avg_surf_min"].round(1).astype(str).values
        + "-"
        + exp_rep["avg_surf_max"].round(1).astype(str).values
    ),
})

def highlight_delta(val):
    if val > 0:
        return "color: green; font-weight: bold"
    elif val < 0:
        return "color: red; font-weight: bold"
    return ""

comparison.style.applymap(highlight_delta, subset=["Delta"])

/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_87858/2975042980.py:46: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  comparison.style.applymap(highlight_delta, subset=["Delta"])


,Condition,Date,Hs (m),Tp (s),Spread,Baseline Score,Experiment Score,Delta,Baseline Surf (ft),Experiment Surf (ft)
0,Cleanest (narrow spread),2003-01-07,3.570000,19.500000,6.200000,9.900000,9.900000,0.000000,7.2-11.4,7.2-11.4
1,Longest period,2020-12-03,1.780000,21.600000,8.000000,9.200000,9.200000,0.100000,3.2-5.2,3.2-5.2
2,Biggest swell,2023-01-05,6.420000,14.800000,8.400000,8.400000,8.200000,-0.100000,10.7-16.0,10.7-16.0
3,Avg swell size,2015-09-08,0.790000,16.299999,13.800000,7.300000,7.500000,0.200000,1.9-2.7,1.9-2.7
4,Avg period,2017-04-01,1.070000,13.300000,11.600000,7.100000,7.100000,0.000000,2.3-3.2,2.3-3.2
5,Avg spread,2009-09-15,1.000000,13.000000,12.400000,6.400000,6.300000,-0.000000,2.2-3.1,2.2-3.1
6,Avg score,2005-12-01,1.260000,11.500000,10.700000,6.200000,6.100000,-0.100000,2.5-3.4,2.5-3.4
7,Messiest (wide spread),2008-01-28,1.280000,9.900000,20.299999,4.100000,3.700000,-0.400000,3.1-4.2,3.1-4.2
8,Shortest period,2000-01-21,0.610000,9.100000,14.000000,3.000000,2.800000,-0.200000,1.2-1.8,1.2-1.8
9,Smallest swell,2023-07-21,0.180000,9.100000,18.400000,1.200000,1.200000,0.000000,0.5-0.5,0.5-0.5


---

## 9. Wind & Tide Config Tuning

Test how wind and tide config parameters affect the **final surf score**.
Uses `calculate_surf_score()` which combines swell, wind, and tide into a single 1-10 score.

We simulate a grid of conditions: a few fixed swell scores paired with
varying wind speeds, wind categories, and tide heights.

In [14]:
from process_data import calculate_surf_score

# Fixed swell scenarios to test against
swell_scenarios = {
    "Small clean swell (score=4)": 4.0,
    "Medium swell (score=6)": 6.0,
    "Pumping swell (score=8)": 8.0,
}

# Wind conditions to sweep
wind_conditions = [
    {"label": "Glassy (3 mph offshore)", "speed": 3.0, "gust": 5.0, "cat": "offshore"},
    {"label": "Light offshore (8 mph)", "speed": 8.0, "gust": 12.0, "cat": "offshore"},
    {"label": "Moderate cross (10 mph)", "speed": 10.0, "gust": 14.0, "cat": "crosshore"},
    {"label": "Moderate onshore (12 mph)", "speed": 12.0, "gust": 18.0, "cat": "onshore"},
    {"label": "Strong onshore (18 mph)", "speed": 18.0, "gust": 25.0, "cat": "onshore"},
]

# Tide conditions to sweep
tide_heights = [-0.5, 0.5, 1.0, 2.0, 3.5, 5.0]


### 9a. Wind Config Experiment

Edit the wind config values below, then re-run to see how the final surf score changes.

In [18]:
wind_exp_config = copy.deepcopy(baseline_surf)

# --- EDIT WIND CONFIG BELOW ---
wind_exp_config["wind"]["speed_floor"] = 5.0          # mph below this = no penalty
wind_exp_config["wind"]["speed_range"] = 13.0         # mph from glass to blown out
wind_exp_config["wind"]["gust_weight"] = 0.3          # 0=ignore gusts, 1=gusts only
wind_exp_config["wind"]["offshore_penalty_weight"] = 0.7
wind_exp_config["wind"]["crosshore_penalty_weight"] = 1.0
wind_exp_config["wind"]["onshore_penalty_weight"] = 1.0
wind_exp_config["wind"]["penalty_min"] = 0.25         # floor multiplier (blown out)
wind_exp_config["final_scoring"]["wind_impact_weight"] = 0.60
# --- EDIT WIND CONFIG ABOVE ---

# Build comparison: baseline vs experiment across all wind/swell combos
rows = []
for swell_label, swell_score in swell_scenarios.items():
    for w in wind_conditions:
        base_final, _, base_wind, _ = calculate_surf_score(
            swell_score, w["speed"], w["gust"], w["cat"], 1.0, baseline_surf
        )
        exp_final, _, exp_wind, _ = calculate_surf_score(
            swell_score, w["speed"], w["gust"], w["cat"], 1.0, wind_exp_config
        )
        rows.append({
            "Swell": swell_label,
            "Wind": w["label"],
            "Baseline Wind Score": base_wind,
            "Exp Wind Score": exp_wind,
            "Baseline Final": base_final,
            "Exp Final": exp_final,
            "Delta": round(exp_final - base_final, 1),
        })

wind_comparison = pd.DataFrame(rows)
wind_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_87858/2291957163.py:35: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  wind_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Wind,Baseline Wind Score,Exp Wind Score,Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),Glassy (3 mph offshore),10.000000,10.000000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),Light offshore (8 mph),8.000000,8.900000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),Moderate cross (10 mph),4.000000,5.200000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),Moderate onshore (12 mph),2.500000,3.200000,2.500000,3.200000,0.700000
4,Small clean swell (score=4),Strong onshore (18 mph),2.500000,2.500000,2.500000,2.500000,0.000000
5,Medium swell (score=6),Glassy (3 mph offshore),10.000000,10.000000,6.000000,6.000000,0.000000
6,Medium swell (score=6),Light offshore (8 mph),8.000000,8.900000,6.000000,6.000000,0.000000
7,Medium swell (score=6),Moderate cross (10 mph),4.000000,5.200000,4.000000,5.200000,1.200000
8,Medium swell (score=6),Moderate onshore (12 mph),2.500000,3.200000,3.300000,3.600000,0.300000
9,Medium swell (score=6),Strong onshore (18 mph),2.500000,2.500000,3.300000,3.300000,0.000000


### 9b. Tide Config Experiment

Edit the tide config values below, then re-run to see how the final surf score changes.

In [10]:
tide_exp_config = copy.deepcopy(baseline_surf)

# --- EDIT TIDE CONFIG BELOW ---
tide_exp_config["tide"]["optimal_height"] = 1.0       # center of Gaussian (ft MLLW)
tide_exp_config["tide"]["sigma"] = 2.5                # width: larger = more tolerant
tide_exp_config["tide"]["penalty_min"] = 0.1          # floor multiplier
tide_exp_config["final_scoring"]["tide_impact_weight"] = 0.60
# --- EDIT TIDE CONFIG ABOVE ---

# Build comparison: baseline vs experiment across all tide/swell combos
rows = []
for swell_label, swell_score in swell_scenarios.items():
    for tide in tide_heights:
        base_final, _, _, base_tide = calculate_surf_score(
            swell_score, 3.0, 5.0, "offshore", tide, baseline_surf
        )
        exp_final, _, _, exp_tide = calculate_surf_score(
            swell_score, 3.0, 5.0, "offshore", tide, tide_exp_config
        )
        rows.append({
            "Swell": swell_label,
            "Tide (ft)": tide,
            "Baseline Tide Score": base_tide,
            "Exp Tide Score": exp_tide,
            "Baseline Final": base_final,
            "Exp Final": exp_final,
            "Delta": round(exp_final - base_final, 1),
        })

tide_comparison = pd.DataFrame(rows)
tide_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_87858/1479117211.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  tide_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Tide (ft),Baseline Tide Score,Exp Tide Score,Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),-0.500000,8.400000,8.400000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),0.500000,9.800000,9.800000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),1.000000,10.000000,10.000000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),2.000000,9.200000,9.200000,4.000000,4.000000,0.000000
4,Small clean swell (score=4),3.500000,6.100000,6.100000,4.000000,4.000000,0.000000
5,Small clean swell (score=4),5.000000,2.800000,2.800000,2.800000,2.800000,0.000000
6,Medium swell (score=6),-0.500000,8.400000,8.400000,6.000000,6.000000,0.000000
7,Medium swell (score=6),0.500000,9.800000,9.800000,6.000000,6.000000,0.000000
8,Medium swell (score=6),1.000000,10.000000,10.000000,6.000000,6.000000,0.000000
9,Medium swell (score=6),2.000000,9.200000,9.200000,6.000000,6.000000,0.000000


### 9c. Combined: Full Scenario Grid

Applies both wind and tide experiment configs simultaneously to see the combined effect.

In [19]:
combined_config = copy.deepcopy(baseline_surf)
combined_config["wind"] = copy.deepcopy(wind_exp_config["wind"])
combined_config["tide"] = copy.deepcopy(tide_exp_config["tide"])
combined_config["final_scoring"] = copy.deepcopy(wind_exp_config["final_scoring"])
combined_config["final_scoring"]["tide_impact_weight"] = tide_exp_config["final_scoring"]["tide_impact_weight"]

# Pick a few representative combos
combo_winds = [wind_conditions[0], wind_conditions[2], wind_conditions[4]]  # glassy, mod cross, strong onshore
combo_tides = [-0.5, 1.0, 3.5]

rows = []
for swell_label, swell_score in swell_scenarios.items():
    for w in combo_winds:
        for tide in combo_tides:
            base_final, _, _, _ = calculate_surf_score(
                swell_score, w["speed"], w["gust"], w["cat"], tide, baseline_surf
            )
            exp_final, _, _, _ = calculate_surf_score(
                swell_score, w["speed"], w["gust"], w["cat"], tide, combined_config
            )
            rows.append({
                "Swell": swell_label,
                "Wind": w["label"],
                "Tide (ft)": tide,
                "Baseline Final": base_final,
                "Exp Final": exp_final,
                "Delta": round(exp_final - base_final, 1),
            })

combined_comparison = pd.DataFrame(rows)
combined_comparison.style.applymap(highlight_delta, subset=["Delta"])


/var/folders/5v/cz1w61mx7hg5nn2hy8ybbf3c0000gn/T/ipykernel_87858/851106371.py:31: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  combined_comparison.style.applymap(highlight_delta, subset=["Delta"])


,Swell,Wind,Tide (ft),Baseline Final,Exp Final,Delta
0,Small clean swell (score=4),Glassy (3 mph offshore),-0.500000,4.000000,4.000000,0.000000
1,Small clean swell (score=4),Glassy (3 mph offshore),1.000000,4.000000,4.000000,0.000000
2,Small clean swell (score=4),Glassy (3 mph offshore),3.500000,4.000000,4.000000,0.000000
3,Small clean swell (score=4),Moderate cross (10 mph),-0.500000,4.000000,4.000000,0.000000
4,Small clean swell (score=4),Moderate cross (10 mph),1.000000,4.000000,4.000000,0.000000
5,Small clean swell (score=4),Moderate cross (10 mph),3.500000,4.000000,4.000000,0.000000
6,Small clean swell (score=4),Strong onshore (18 mph),-0.500000,2.500000,2.500000,0.000000
7,Small clean swell (score=4),Strong onshore (18 mph),1.000000,2.500000,2.500000,0.000000
8,Small clean swell (score=4),Strong onshore (18 mph),3.500000,2.500000,2.500000,0.000000
9,Medium swell (score=6),Glassy (3 mph offshore),-0.500000,6.000000,6.000000,0.000000
